In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
import cv2
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Sequential, Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D,Dropout
import tensorflow as tf
import splitfolders
import pandas as pd
import glob
from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go
import itertools
import plotly.express as px

In [15]:
#data directory
data_dir = pathlib.Path("D:\\python\\meow\\pruby\\ds\\pista\\Pistachio_Image_Dataset")
#print all folders in the data directory
print("Folders in the data directory:")
for item in data_dir.iterdir():
    print(item)



Folders in the data directory:
D:\python\meow\pruby\ds\pista\Pistachio_Image_Dataset\Pistachio_16_Features_Dataset
D:\python\meow\pruby\ds\pista\Pistachio_Image_Dataset\Pistachio_28_Features_Dataset
D:\python\meow\pruby\ds\pista\Pistachio_Image_Dataset\Pistachio_Image_Dataset
D:\python\meow\pruby\ds\pista\Pistachio_Image_Dataset\Pistachio_Image_Dataset_Request.txt


In [25]:
#all of the images
images = glob.glob("D:\\python\\meow\\pruby\\ds\\pista\\Pistachio_Image_Dataset\\Pistachio_Image_Dataset\\*\\*.jpg")
#number of images
print("Number of images: ", len(images))

images = pd.Series(images)


Number of images:  2148


In [27]:
df = pd.DataFrame()

#generate field with Filename
df['Filename'] = images.map(lambda img_name: img_name.split('\\')[-1])

#generate field with ClassID
df['ClassID'] = images.map(lambda img_name: img_name.split('\\')[-2])

df.head()

,Filename,ClassID
0,kirmizi (1).jpg,Kirmizi_Pistachio
1,kirmizi (10).jpg,Kirmizi_Pistachio
2,kirmizi (11).jpg,Kirmizi_Pistachio
3,kirmizi (12).jpg,Kirmizi_Pistachio
4,kirmizi (13).jpg,Kirmizi_Pistachio


In [29]:
class_id_distributionTotal = df['ClassID'].value_counts()
class_id_distributionTotal.head(10)

Kirmizi_Pistachio    1232
Siirt_Pistachio       916
Name: ClassID, dtype: int64

In [31]:
#data distribution
fig = px.bar(class_id_distributionTotal, x=class_id_distributionTotal.index, y=class_id_distributionTotal.values, color=class_id_distributionTotal.index)
fig.update_layout(title_text='Data distribution', xaxis_title="Class ID", yaxis_title="Number of images")
fig.show()

In [32]:
#percentage of data distribution in a pie chart
fig = px.pie(class_id_distributionTotal, values=class_id_distributionTotal.values, names=class_id_distributionTotal.index, title='Data distribution')
fig.show()


In [36]:
#split data into train, validation and test sets
splitfolders.ratio("D:\\python\\meow\\pruby\\ds\\pista\\Pistachio_Image_Dataset\\Pistachio_Image_Dataset", output="output", seed=1337, ratio=(.8, 0.1,0.1), group_prefix=None) # default values

#show the folders in the output directory
print("Folders in the output directory:")
for item in pathlib.Path("output").iterdir():
    print(item)
#show items in these directories
print("Items in the train directory:")
for item in pathlib.Path("output\\train").iterdir():
    print(item)
print("Items in the val directory:")
for item in pathlib.Path("output\\val").iterdir():
    print(item)
print("Items in the test directory:")
for item in pathlib.Path("output\\test").iterdir():
    print(item)

Copying files: 2148 files [00:11, 193.28 files/s]

Folders in the output directory:
output\test
output\train
output\val
Items in the train directory:
output\train\Kirmizi_Pistachio
output\train\Pistachio_16_Features_Dataset
output\train\Pistachio_28_Features_Dataset
output\train\Pistachio_Image_Dataset
output\train\Siirt_Pistachio
Items in the val directory:
output\val\Kirmizi_Pistachio
output\val\Pistachio_16_Features_Dataset
output\val\Pistachio_28_Features_Dataset
output\val\Pistachio_Image_Dataset
output\val\Siirt_Pistachio
Items in the test directory:
output\test\Kirmizi_Pistachio
output\test\Pistachio_16_Features_Dataset
output\test\Pistachio_28_Features_Dataset
output\test\Pistachio_Image_Dataset
output\test\Siirt_Pistachio


In [37]:
train_dir = pathlib.Path("output\\train")
val_dir = pathlib.Path("output\\val")
test_dir = pathlib.Path("output\\test")
class_names_train = os.listdir(train_dir)
class_names_val = os.listdir(val_dir)
class_names_test = os.listdir(test_dir)


In [39]:
train_images = glob.glob("output\\train\\*\\*.jpg")
total_train_images = train_images
print("Number of train images: ", len(train_images))
test_images = glob.glob("output\\test\\*\\*.jpg")
total_test_images = test_images
print("Number of test images: ", len(test_images))
val_images = glob.glob("output\\val\\*\\*.jpg")
total_val_images = val_images
print("Number of val images: ", len(val_images))


Number of train images:  1717
Number of test images:  217
Number of val images:  214


In [42]:
#chart for the data distribution of train, val and test sets
random_x = [len(train_images), len(val_images), len(test_images)]
names = ['Train', 'Validation', 'Test']
fig = go.Figure(data=[go.Pie(labels=names, values=random_x, hole=.3)])
fig.update_layout(title_text='Data distribution', xaxis_title="Class ID", yaxis_title="Number of images")
fig.show()

In [43]:
train_image_names = pd.Series(train_images)
train_df = pd.DataFrame()

#generate field with Filename
train_df['Filename'] = train_image_names.map(lambda img_name: img_name.split('\\')[-1])

#generate field with ClassID
train_df['ClassID'] = train_image_names.map(lambda img_name: img_name.split('\\')[-2])

train_df.head()


,Filename,ClassID
0,kirmizi (10).jpg,Kirmizi_Pistachio
1,kirmizi (11).jpg,Kirmizi_Pistachio
2,kirmizi (12).jpg,Kirmizi_Pistachio
3,kirmizi (13).jpg,Kirmizi_Pistachio
4,kirmizi (15).jpg,Kirmizi_Pistachio


In [44]:
class_id_distribution_Train = train_df['ClassID'].value_counts()
class_id_distribution_Train.head(10)

Kirmizi_Pistachio    985
Siirt_Pistachio      732
Name: ClassID, dtype: int64